In [2]:
import functools

import numpy as np
import tensorflow as tf

In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [4]:
np.set_printoptions(precision=3, suppress=True)

In [5]:
LABEL_COLUMN = "survived"
LABELS = [0, 1]

In [6]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


In [7]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print(f"{key:20s}: {value.numpy()}")

In [8]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [45. 25. 39. 30. 21.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [0 1 0 0 0]
fare                : [26.55  30.    13.     7.225  7.775]
class               : [b'First' b'Second' b'Second' b'Third' b'Third']
deck                : [b'B' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [9]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'female' b'female' b'female']
age                 : [26. 50. 28. 31. 28.]
n_siblings_spouses  : [0 0 1 1 1]
parch               : [0 0 0 1 0]
fare                : [18.788 13.    15.5   26.25  24.   ]
class               : [b'Third' b'Second' b'Third' b'Second' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Queenstown' b'Southampton' b'Cherbourg']
alone               : [b'y' b'y' b'n' b'n' b'n']


In [10]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [ 0.83 36.   28.   28.   18.  ]
n_siblings_spouses  : [0 0 0 0 1]
class               : [b'Second' b'First' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'B' b'unknown' b'unknown' b'unknown']
alone               : [b'n' b'n' b'y' b'y' b'n']


In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS, column_defaults=DEFAULTS)

show_batch(temp_dataset)

age                 : [28.  9. 19. 60. 40.]
n_siblings_spouses  : [0. 5. 1. 1. 1.]
parch               : [0. 2. 0. 1. 0.]
fare                : [13.    46.9   26.    79.2    9.475]


In [12]:
example_batch, labels_batch = next(iter(temp_dataset))

In [13]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [14]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[28.     0.     0.     7.229]
 [65.     0.     0.    26.55 ]
 [35.     0.     0.    26.288]
 [26.     0.     0.    18.788]
 [59.     0.     0.    13.5  ]]

[1 0 1 1 0]


In [15]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'female' b'male']
age                 : [ 2. 28. 20. 19. 36.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [1 0 0 2 0]
fare                : [10.462  8.05   7.229 26.283  7.496]
class               : [b'Third' b'Third' b'Third' b'First' b'Third']
deck                : [b'G' b'unknown' b'unknown' b'D' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'y']


In [16]:
example_batch, labels_batch = next(iter(temp_dataset))

In [17]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names

    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features["numeric"] = numeric_features

        return features, labels

In [18]:
NUMERIC_FEATURES = ["age", "n_siblings_spouses", "parch", "fare"]

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [19]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'female']
class               : [b'Third' b'Third' b'Second' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'unknown']
alone               : [b'y' b'n' b'y' b'n' b'y']
numeric             : [[28.     0.     0.     8.458]
 [25.     1.     0.     7.775]
 [23.     0.     0.    10.5  ]
 [28.     8.     2.    69.55 ]
 [38.     0.     0.    80.   ]]


In [20]:
example_batch, labels_batch = next(iter(packed_train_data))

In [21]:
import pandas as pd

desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [22]:
MEAN = np.array(desc.T["mean"])
STD = np.array(desc.T["std"])

In [23]:
def normalize_numeric_data(data, mean, std):
    return (data-mean) / std

In [31]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column("numeric", normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x0000023235889B40>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [32]:
example_batch["numeric"]

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  0.   ,  0.   ,  7.55 ],
       [26.   ,  0.   ,  0.   , 18.788],
       [28.   ,  0.   ,  0.   ,  6.858],
       [27.   ,  1.   ,  0.   , 53.1  ],
       [28.   ,  0.   ,  0.   ,  7.896]], dtype=float32)>

In [33]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_column)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479, -0.492],
       [-0.29 , -0.474, -0.479, -0.286],
       [-0.13 , -0.474, -0.479, -0.504],
       [-0.21 ,  0.395, -0.479,  0.343],
       [-0.13 , -0.474, -0.479, -0.485]], dtype=float32)

In [34]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [35]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [36]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [38]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [39]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [40]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13  -0.474
 -0.479 -0.492  1.     0.   ]


In [41]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [42]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [43]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 2s 8ms/step - loss: 0.4970 - accuracy: 0.7432
Epoch 2/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4201 - accuracy: 0.8134
Epoch 3/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4008 - accuracy: 0.8309
Epoch 4/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3915 - accuracy: 0.8262
Epoch 5/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3853 - accuracy: 0.8357
Epoch 6/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3662 - accuracy: 0.8469
Epoch 7/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3634 - accuracy: 0.8405
Epoch 8/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3549 - accuracy: 0.8437
Epoch 9/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3506 - accuracy: 0.8565
Epoch 10/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3434 - accuracy: 0.8596

In [44]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 2ms/step - loss: 0.4880 - accuracy: 0.8371


Test Loss 0.4880037009716034, Test Accuracy 0.8371211886405945


In [45]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

53/53 [==============================] - 0s 1ms/step
Predicted survival: 12.12%  | Actual outcome:  DIED
Predicted survival: 99.99%  | Actual outcome:  DIED
Predicted survival: 68.26%  | Actual outcome:  DIED
Predicted survival: 64.52%  | Actual outcome:  DIED
Predicted survival: 94.32%  | Actual outcome:  SURVIVED
